# Dogs Breed Classifier using CNN

## Importing Libraries

In [54]:
import tensorflow as tf 
from keras.preprocessing.image import ImageDataGenerator

In [55]:
tf.__version__

'1.14.0'

## Data Preprocessing

### Generating Images for Training Set

In [56]:
trainDataGen = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True 
)

trainingSet = trainDataGen.flow_from_directory(
    '../data/training-set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 4374 images belonging to 26 classes.


### Writing breed classes in json file

In [57]:
import json 

dog_classes = trainingSet.class_indices
dog_classes_json = json.dumps(dog_classes)

with open('dog_classes.json', 'w+') as file:
    file.write(dog_classes_json)

### Generating images for Test set

In [58]:
testDataGen = ImageDataGenerator(
    rescale = 1.0 / 255.0
)

testSet = testDataGen.flow_from_directory(
    '../data/test-set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 260 images belonging to 26 classes.


## Building The CNN

In [59]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, Activation
from keras.models import Sequential

### Defining Neural Network Architecture

In [60]:
model = Sequential()

### First Convolutional Layer

In [61]:
model.add(Conv2D(32, (3, 3), input_shape=( 64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

### Second Convolution Layer

In [62]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

### Third Convolutional Layer

In [63]:
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

### Flattening

In [64]:
model.add(Flatten()) 

### Full Connection

In [65]:
model.add(Dense(64))
model.add(Activation('relu'))

### Output Layer

In [66]:
model.add(Dense(26))
model.add(Activation('softmax'))

In [67]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 29, 29, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 64)       

### Compiling the CNN

In [68]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Training Model on Train data

In [69]:
model.fit(x = trainingSet, validation_data = testSet, epochs = 30)

Epoch 1/30
137/137 [==============================] - 35s 253ms/step - loss: 3.2492 - accuracy: 0.0560 - val_loss: 3.1865 - val_accuracy: 0.0654
Epoch 2/30
137/137 [==============================] - 38s 277ms/step - loss: 3.1356 - accuracy: 0.1001 - val_loss: 3.8447 - val_accuracy: 0.1000
Epoch 3/30
137/137 [==============================] - 59s 429ms/step - loss: 3.0366 - accuracy: 0.1253 - val_loss: 2.9732 - val_accuracy: 0.1192
Epoch 4/30
137/137 [==============================] - 56s 408ms/step - loss: 2.9711 - accuracy: 0.1429 - val_loss: 2.8415 - val_accuracy: 0.1423
Epoch 5/30
137/137 [==============================] - 38s 280ms/step - loss: 2.9181 - accuracy: 0.1509 - val_loss: 2.4946 - val_accuracy: 0.1615
Epoch 6/30
137/137 [==============================] - 38s 279ms/step - loss: 2.8412 - accuracy: 0.1655 - val_loss: 3.1949 - val_accuracy: 0.1769
Epoch 7/30
137/137 [==============================] - 38s 277ms/step - loss: 2.7830 - accuracy: 0.1806 - val_loss: 2.2932 - val_ac

### Saving Model to binary file

In [70]:
model.save('model')